In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import pandas as pd
CLEAN_DATA_DIR = "../data/clean/"

In [8]:
def encoder(data):
  col = [col for col in data.columns if -1 in data[col].values]
  # One-hot encode categorical columns
  encoder = OneHotEncoder(sparse_output=False, drop=None)
  encoded = encoder.fit_transform(data[col])

  # Convert to DataFrame and combine with numerical features
  encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(col), index=data.index)
  numerical_df = data.drop(columns=col)

  # Combine numerical and encoded categorical data
  processed_df = pd.concat([numerical_df, encoded_df], axis=1)
  return processed_df


def encode_filler(data):
  na_col = ['Q06_11.0', 'Q11_5.0', 'Q06_10.0', 'Q11_9.0', 'Q06_9.0', 'Q07_-1.0', 'Q06_2.0', 'Q01', 'Q11_14.0', 'Q06_5.0',
              'Q11_2.0', 'Q06_4.0', 'Q08_-1.0', 'Q07_4.0', 'Q08_2.0', 'Q07_0.0', 'q02', 'Q06_8.0', 'Q07_2.0', 'Q11_3.0', 'Q03',
              'Q11_4.0', 'q23', 'Q11_7.0', 'Q11_13.0', 'Q06_1.0', 'Q19_2.0', 'Q06_-1.0', 'Q11_-1.0', 'Q11_10.0', 'q05', 'Q07_1.0',
              'Q11_12.0', 'Q19_-1.0', 'Q06_7.0', 'Q11_1.0', 'Q19_1.0', 'Q11_8.0', 'Q08_1.0', 'Q06_0.0', 'q03', 'Q06_3.0', 'q09',
              'Q07_3.0', 'Q11_11.0', 'Q06_6.0']

  for col in na_col:
      if col not in data.columns:
          data[col] = 0  # Assign 0
  
  filled_data = data[na_col]
  return filled_data

In [ ]:
def predict_ratings_SVM(train_data):

    train_x = train_data.drop(columns=['psu_hh_idcode', 'subjectivePoverty_rating'])

    # Combine numerical and encoded categorical data
    processed_train = encoder(train_x)
    processed_train = encode_filler(processed_train)

    y = train_data['subjectivePoverty_rating']
    X = processed_train

  
    # Scale the features
    # scaler = StandardScaler()
    # X_train_scaled = scaler.fit_transform(X)
    
   # GridSearch CV
    param_grid = {
        'C': [0.5, 1, 10, 100],
        'gamma': ['scale', 0.1, 0.01, 0.001],
        'kernel': ['rbf']
    }

    optimal_params = GridSearchCV(SVC(probability=True, random_state=42), param_grid, n_jobs=-1, cv=5, scoring='neg_log_loss')
    
    # Fit the model
    optimal_params.fit(X, y)
    return optimal_params.best_estimator_

In [10]:
RESULT_DATA_DIR = "../data/model_result/"
train_data = pd.read_csv(os.path.join(CLEAN_DATA_DIR, "TRAIN_MERGED_UNFILLED.csv"))

train_data = train_data.drop(columns=['hhid'])
train_A, train_B = train_test_split(train_data, test_size=0.25, stratify=train_data['subjectivePoverty_rating'], random_state=42)

model = predict_ratings_SVM(train_A)

In [11]:
train_B_X = train_B.drop(columns=['subjectivePoverty_rating'])
# display(train_B_X)

def predictratings_SVM(model, train_B_X):
    test_ids = train_B_X['psu_hh_idcode']
    train_B_X = encoder(train_B_X)
    train_B_X = encode_filler(train_B_X)
    preds_proba = model.predict_proba(train_B_X)
    print(log_loss(train_B['subjectivePoverty_rating'], preds_proba))
    output_df = pd.DataFrame(preds_proba, columns=[f'subjective_poverty{i+1}' for i in range(preds_proba.shape[1])])
    output_df.insert(0, 'psu_hh_idcode', test_ids.values)  # Insert the ID column at the start
    return output_df

pred = predictratings_SVM(model, train_B_X)
display(pred)

# train_B_X = train_B.drop(columns=['subjectivePoverty_rating'])
# display(train_B_X)

def predictratings_SVM(model, train_B_X):
    test_ids = train_B_X['psu_hh_idcode']
    train_B_X = encoder(train_B_X)
    train_B_X = encode_filler(train_B_X)
    preds_proba = model.predict_proba(train_B_X)
    print(log_loss(train_B['subjectivePoverty_rating'], preds_proba))
    output_df = pd.DataFrame(preds_proba, columns=[f'subjective_poverty{i+1}' for i in range(preds_proba.shape[1])])
    output_df.insert(0, 'psu_hh_idcode', test_ids.values)  # Insert the ID column at the start
    return output_df

pred = predictratings_SVM(model, train_B_X)
display(pred)

#1.9505287965222604 - scale X



1.9505287965222604


,psu_hh_idcode,subjective_poverty1,subjective_poverty2,subjective_poverty3,subjective_poverty4,subjective_poverty5,subjective_poverty6,subjective_poverty7,subjective_poverty8,subjective_poverty9,subjective_poverty10
0,125_11_1,0.044408,0.105230,0.191436,0.223640,0.196458,0.117587,0.074277,0.036250,0.008682,0.002032
1,129_9_1,0.035467,0.084355,0.153981,0.213245,0.218274,0.162251,0.079833,0.042062,0.008984,0.001548
2,800_8_1,0.034335,0.076847,0.161290,0.221429,0.210482,0.145850,0.087221,0.050363,0.010334,0.001850
3,472_1_1,0.027304,0.059134,0.093918,0.175930,0.226580,0.191850,0.135737,0.072433,0.012605,0.004508
4,309_11_1,0.034067,0.074956,0.160364,0.213185,0.220444,0.160751,0.088881,0.038577,0.007069,0.001707
...,...,...,...,...,...,...,...,...,...,...,...
1329,588_5_1,0.043784,0.101356,0.181252,0.223882,0.203168,0.126929,0.073547,0.035554,0.008497,0.002031
1330,566_4_1,0.033804,0.087128,0.170747,0.206867,0.214966,0.156489,0.079781,0.040154,0.007755,0.002308
1331,220_7_1,0.033846,0.073728,0.143525,0.215536,0.220762,0.169334,0.091817,0.040252,0.008797,0.002404
1332,144_5_2,0.044202,0.072819,0.165075,0.190909,0.195553,0.162757,0.108086,0.049431,0.008884,0.002283


1.9505287965222604


,psu_hh_idcode,subjective_poverty1,subjective_poverty2,subjective_poverty3,subjective_poverty4,subjective_poverty5,subjective_poverty6,subjective_poverty7,subjective_poverty8,subjective_poverty9,subjective_poverty10
0,125_11_1,0.044408,0.105230,0.191436,0.223640,0.196458,0.117587,0.074277,0.036250,0.008682,0.002032
1,129_9_1,0.035467,0.084355,0.153981,0.213245,0.218274,0.162251,0.079833,0.042062,0.008984,0.001548
2,800_8_1,0.034335,0.076847,0.161290,0.221429,0.210482,0.145850,0.087221,0.050363,0.010334,0.001850
3,472_1_1,0.027304,0.059134,0.093918,0.175930,0.226580,0.191850,0.135737,0.072433,0.012605,0.004508
4,309_11_1,0.034067,0.074956,0.160364,0.213185,0.220444,0.160751,0.088881,0.038577,0.007069,0.001707
...,...,...,...,...,...,...,...,...,...,...,...
1329,588_5_1,0.043784,0.101356,0.181252,0.223882,0.203168,0.126929,0.073547,0.035554,0.008497,0.002031
1330,566_4_1,0.033804,0.087128,0.170747,0.206867,0.214966,0.156489,0.079781,0.040154,0.007755,0.002308
1331,220_7_1,0.033846,0.073728,0.143525,0.215536,0.220762,0.169334,0.091817,0.040252,0.008797,0.002404
1332,144_5_2,0.044202,0.072819,0.165075,0.190909,0.195553,0.162757,0.108086,0.049431,0.008884,0.002283


psu_hh_idcode	subjective_poverty1	subjective_poverty2	subjective_poverty3	subjective_poverty4	subjective_poverty5	subjective_poverty6	subjective_poverty7	subjective_poverty8	subjective_poverty9	subjective_poverty10
0	125_11_1	0.025525	0.059814	0.087623	0.220639	0.233748	0.182541	0.121753	0.053184	0.011969	0.003203
1	129_9_1	0.037075	0.058612	0.088723	0.214832	0.184983	0.169812	0.163631	0.064149	0.014398	0.003784
2	800_8_1	0.027463	0.057546	0.080168	0.217410	0.228223	0.184984	0.132869	0.054868	0.012673	0.003796
3	472_1_1	0.034906	0.057764	0.085594	0.217024	0.193675	0.172937	0.157929	0.062606	0.014007	0.003559
4	309_11_1	0.027289	0.057630	0.078263	0.222137	0.226802	0.182085	0.132389	0.057131	0.012434	0.003839
...	...	...	...	...	...	...	...	...	...	...	...
1329	588_5_1	0.026330	0.058408	0.083609	0.219034	0.233413	0.183294	0.125962	0.054159	0.012287	0.003504
1330	566_4_1	0.036910	0.058539	0.088463	0.214998	0.185711	0.170098	0.163133	0.064025	0.014363	0.003761
1331	220_7_1	0.031674	0.057190	0.081302	0.220262	0.208272	0.177602	0.146896	0.059892	0.013415	0.003495
1332	144_5_2	0.024264	0.065964	0.090802	0.210098	0.245841	0.185363	0.109233	0.054836	0.011278	0.002322
1333	18_7_1	0.034787	0.057585	0.085540	0.217065	0.194416	0.173364	0.157189	0.062621	0.013908	0.003525
